<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/PR%C3%80CTICA_GUIADA/M%C3%88TODES_COR_PROGRAMA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Partim de la EDP:


$c_vρ\frac{∂T}{∂t}=∇⋅(κ{\vec\nabla T})+P_{ext}$

Després de normalitzar i reduïr els paràmetres al mínim obtenim:

$\frac{\partial\tilde{T}}{\partial\tilde{t}}=\frac{\partial^2\tilde{T}}{\partial\tilde{x}^2}+1$



Discretitzem via Euler explícit amb la derivada per la dreta:

$\frac{T_{i+1}^{(n)}-T_i^{(n)}}{Δt}=1+\frac{T_i^{(n+1)}-2T_i^{(n)}+T_i^{(n-1)}}{(Δx)^2}$

Aïllem $T_{i+1}^{(n)}$:

$T_{i+1}^{(n)}=Δt+\alpha T_{i}^{(n-1)}+βT_{i}^{(n)} + α T_{i}^{(n+1)}$

On: $\ α=\frac{Δt}{\Delta x^2},\ β=1-\frac{2\Delta t}{\Delta x^2}$

Això es pot escriure en forma matricial i calcular tots els punts espaials de la malla a temps i+1 a partir dels de i. Ja que tenim tots els punts de i=0 de les condicions de contorn

I les condicions de contorn són:

*   $T(0,x)=36,5\ ℃$
*   $\frac{\partial T(t,\ 0)}{\partial t}=\frac{\partial T(t,\ 2\ cm)}{\partial t}=0$



In [8]:
#EULER EXPLÍCIT

import numpy as np
import matplotlib.pyplot as plt

N= 101   # Mallat espaial
k_1= 0.51*N**2  # disc temps 1
k_2= 0.49*N**2  # disc temps 2
k_3= 0.25*N**2  # disc temps 3

x_0= 0.01
x_f= 2/x_0
x_i= 0


alpha_var= 7/49807075
t_f= 0.025
t_i= 0
t_0= 711.5296429  #x_0**2/alpha_var

#Variacions de les variables (els tres delta t corresponen als 3 de l'enunciat)
dt_1= t_f/k_1
dt_2= None
dt_3= None
dx= x_f/N

alpha= dt_1/(dx**2)
beta= 1-2*dt_1/(dx**2)

matriu= np.array([])

T_0= 337.1428572
T_init= 36.5/T_0   #Temperatura inicial
T_i= np.ones((N,1))*T_init  #Vector columna de les T_i per cada n (punt de l'espai)
print(T_i)
deltat= np.ones((N,1))*dt_1 #Vector columna de Δt per sumar a l'expressió

#Generem la matriu
for i in range(N):
  fila= [0]*N
  fila[i]= alpha
  if i< N-1:
    fila[i+1] = beta
  if i < N-2:
    fila[i+2] = alpha
  #print(fila, len(fila))
  matriu= np.append(matriu,fila)

print(matriu)


#EULER CÀLCUL

for i in range(int(k_1)):
  T_i= np.matmul(matriu,T_i)+deltat
  t_i= t_i + dt_1
  x_i= x_i + dx


[[0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10826271]
 [0.10

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 101 is different from 10201)